# Composite analysis with Monte Carlo methods: an example with cosmic rays and clouds#

## Benjamin A. Laken & Jasa Čalogović ##

This is an IPython notebook version of [Laken & Calogovic, 2013, doi: 10.1051/swsc/2013051](http://www.swsc-journal.org/articles/swsc/full_html/2013/01/swsc130020/swsc130020.html), published in the **Journal of Space Weather and Space Climate**. Originally, this work was supported by [IDL code](http://www.files.benlaken.com/documents/Laken_Calogovic_2013/), succeeded by this notebook.

Code in Python 3, by [B. Laken](http://www.benlaken.com)

### Abstract ###

The composite (superposed epoch) analysis technique has been frequently employed to examine a hypothesized link between solar activity and the Earth’s atmosphere, often through an investigation of Forbush decrease (Fd) events (sudden high-magnitude decreases in the flux cosmic rays impinging on the upper-atmosphere lasting up to several days). This technique is useful for isolating low-amplitude signals within data where background variability would otherwise obscure detection. The application of composite analyses to investigate the possible impacts of Fd events involves a statistical examination of time-dependent atmospheric responses to Fds often from aerosol and/or cloud datasets. Despite the publication of numerous results within this field, clear conclusions have yet to be drawn and much ambiguity and disagreement still remain. In this paper, we argue that the conflicting findings of composite studies within this field relate to methodological differences in the manner in which the composites have been constructed and analyzed. Working from an example, we show how a composite may be objectively constructed to maximize signal detection, robustly identify statistical significance, and quantify the lower-limit uncertainty related to hypothesis testing. Additionally, we also demonstrate how a seemingly significant false positive may be obtained from non-significant data by minor alterations to methodological approaches.

## 1. Introduction ##

The composite analysis technique, also referred to as a superposed epoch analysis, and conditional sampling, is used in numerous fields including geostatistics, fluid dynamics, and plasma physics. It has been frequently employed to examine a hypothesized link between atmospheric properties and sudden decreases in cosmic ray intensity over daily timescales (Forbush decreases). The composite technique is useful for isolating low-amplitude signals within data where background variability would otherwise obscure detection ([Chree 1913](http://rsta.royalsocietypublishing.org/content/212/484-496/75); [1914](http://adsabs.harvard.edu/abs/1914RSPTA.213..245C)). Composite analyses rely on selecting subsets of data, wherein key points in time can be identified based on some criteria, e.g., the occurrence of unusual or extreme events in one or more datasets ([Chree 1913](http://rsta.royalsocietypublishing.org/content/212/484-496/75); [1914](http://adsabs.harvard.edu/abs/1914RSPTA.213..245C); [Forbush et al. 1983](http://link.springer.com/article/10.1007%2FBF00145551)). Through the accumulation and averaging of successive key events, the stochastic background variability may be reduced to a point where low-amplitude signals become identifiable.

To exemplify the composite methodology and how it may be applied to isolate low-amplitude signals, we present the following example. Imagine a regularly sampled time series, $Xi$, where $i = 400$ time units (for ease we shall refer to the these time units as days). As with real-world data, this time series may contain a stochastic (noise) component ($Ni$), and a dynamically determined component ($Di$). Figure 1a shows such a time series, where Di is represented by the addition of two oscillations of differing periods, and Ni is represented by Gaussian (white) noise; the values of both $Ni$ and $Di$ range from 0 to 1. To $Xi$, we have also added a small signal component, $Si$, a regular 5-day deviation of 0.3 units amplitude, repeating at time intervals of 40 days throughout the 400 day time period, so that $Xi = Ni + Di + Si$.

In [ ]:
# Load required packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from tqdm import tqdm
from collections import namedtuple
from scipy.stats import linregress
import datetime as dt
#from profilehooks import profile # a profiler for testing
from bokeh.plotting import figure, show, output_notebook, vplot
from bokeh.io import gridplot
from bokeh.models import HoverTool, Span, Range1d, LinearAxis
from bokeh.models.sources import ColumnDataSource
%matplotlib inline
output_notebook()

In [ ]:
# Synthetic time-series with a periodic signal (smaller than noise) to demonstrate the use of composites

tpoints = 400  # Lenght of time-series (unitless)
isubs = np.array(range(tpoints))/50.

# Deterministic component
di =((np.sin(isubs) * 0.5) + (np.sin(np.pi*isubs)*0.3))
di = di + np.abs(np.min(di)) # Make all values positive
di = di / np.max(di)         # Normalize values

# Stochastic component 
ni = np.random.random(tpoints) -0.5  # random values from 0 - 1, shifted by -0.5

# Signal component
si = np.zeros(tpoints)
signal = dict(zip([-2, -1, 0, 1, 2],[-0.333, -0.666, -1., -0.666, -0.333]))
for index in range(20,tpoints,20):
    for n in range(-2,3):
        si[index + n] = signal[n]
si = si * 0.3 # Reduce the 'signal' size

# Combine the componenets into a synthetic time-series
xi = di + ni + si

df_syn = pd.DataFrame(np.c_[xi, ni, di, si], columns=["xi", "ni", "di", "si"])
#df_syn.head()

In [ ]:
# Bokeh interactive plot of Figure 1
TOOLS = "pan, wheel_zoom, box_zoom, reset"

p1a = figure(width=400, plot_height=300, title=None, tools=TOOLS)
p1a.line(range(tpoints),df_syn.xi +1.9, legend="Xi",color='black')
p1a.line(range(tpoints),df_syn.di + 0.9, legend="Di",color='red')
p1a.line(range(tpoints),df_syn.si + 0.8, legend="Si",color='blue')
p1a.line(range(tpoints),df_syn.ni, legend="Ni",color='green')
p1a.yaxis.visible = None

p1b = figure(width=400, plot_height=300, title=None, x_range=p1a.x_range, tools=TOOLS)
p1b.line(range(tpoints),df_syn.xi,legend="Xi",color='black')
p1b.line(range(tpoints),pd.rolling_mean(df_syn.xi, window=21, center=True),
        legend="21-day smooth",color='red')
p1b.xaxis.axis_label = "Time (unitless)"
p1b.xaxis.axis_label_text_font_size = '10'
p1b.yaxis.axis_label = "Xi (unitless)"
p1b.yaxis.axis_label_text_font_size = '10'

p1c = figure(width=400, plot_height=300, title=None, tools=TOOLS)
xi_anom = xi - pd.rolling_mean(df_syn.xi, window=21, center=True)
tmp=[]
for index in range(20,tpoints,20):
    tmp.append(xi_anom[index-10:index+11])
tmp = np.array(tmp)
for n in range(len(tmp)):
    p1c.line(range(-10,11),tmp[n,:],color="black",alpha=0.5,legend="Ai, composite matrix (n,t)")
p1c.yaxis.axis_label = "δ Xi (unitless)"
p1c.yaxis.axis_label_text_font_size = '10'

p1d = figure(width=400, plot_height=300, title=None, y_range=p1c.y_range, x_range=p1c.x_range, tools=TOOLS)
stdev = tmp.std(axis=0)
error = stdev/np.sqrt(len(tmp))
mvals = tmp.mean(axis=0)
p1d.circle(range(-10,11), mvals, legend="Ct μ")
p1d.line(range(-10,11), mvals+error,legend="Ct S.E.M")
p1d.line(range(-10,11), mvals-error)

p1d.segment(range(-10, 11), mvals+stdev, range(-10, 11), mvals-stdev, color="black",legend="σ")

p1d.xaxis.axis_label = "Days since signal"
p1d.xaxis.axis_label_text_font_size = '10'
p1d.yaxis.axis_label = "δ Xi (unitless)"
p1d.yaxis.axis_label_text_font_size = '10'

fig1 = gridplot([[p1a,p1c],[p1b,p1d]])
show(fig1)

**Figure 1**: (a) Top left: Fictional time series $Xi$, comprised of deterministic ($Di$), stochastic ($Ni$), and a low-amplitude repeating signal ($Si$) components. $Di$ and $Ni$ range from 0.0 to 1.0, while $Si$ has an amplitude of 0.3. (b) Bottom left: Fi, a 21-point smooth (box-car running mean) of $Xi$. By subtracting $Fi$ from $Xi$ a high-pass filtered dataset $Ai$ is produced. (c) Top right: A composite matrix of events from $Ai$, where rows = $n$, the number of repeating signals in $Si$ (a composite event), and columns = $t$, the number of days since the composite event. (d) Bottom left: $Ct$, the composite means of $Ai$, the standard error of the mean (SEM).

Through the successive averaging of events in the composite methodology we may isolate a low-amplitude signal component from the time series. Before doing this, it is beneficial to attempt to remove variations in $Xi$ that are unconnected to the $Si$ component and may reduce our signal-to-noise ratio (SNR). In reality we may have limited knowledge of the properties of a potential signal component within a dataset. If we suppose the signal we are testing for has an upper-limit length shorter than 7 days, we may set a filter length of three times our maximum expected signal length (e.g., 21 days) to use as a high-pass filter. This may eliminate some noise concerns while leaving our signal unaffected (potential loss of signal from filtering, and noise reduction in composites is discussed in later sections). We then calculate $Fi$, a smooth (running mean) of our dataset with a width set by our expected signal (in our example it is 21-days). The values of $Fi$ (thick line) are shown with $Xi$ in Figure 1b.

By subtracting $Fi$ from $Xi$ we obtain $Ai$, a high-pass filtered dataset, which we shall refer to throughout this work as an anomaly. With prior knowledge of the occurrence times of the signal we may construct a composite matrix, $Mjt$, where $j = 1, … , n$ enumerates the $n$ composite events (matrix rows), and $t$ is the time dimension (matrix columns). We present the composite matrix of $Ai$ in Figure 1c, over a $t = \pm20$ period. In any one of the 10 events in the composite matrix it would be highly difficult to objectively determine the presence of a signal at $t_{0}$. However, by averaging the 10 events of the matrix together to create a composite mean

Equation 1.    $C_{t}=\frac 1 n \displaystyle\sum_{j=0}^{n} M_{jt}$

the noise is reduced by the square root of the number of events

Equation 2.    $\Delta C_{t}=\frac\sigma {\sqrt[]{n}}$

where $\delta C_{t}$ indicates the standard error of the mean (SEM) at $t$ and $\sigma$ is the sample standard deviation of $C_{t}$. As the noise diminishes the signal has an increased chance of being identified. For each time step t, the mean and SEM of the matrix may be calculated using Equations 1 and 2 respectively; these data are presented in Figure 1d, and indeed show the successful isolation of a signal of approximately 0.3 amplitude centered on $t_{0}$. This example and all those presented in this manuscript relate only to one particular statistic, the sample mean, however, any other statistic may also be used. While the composite approach appears straightforward, inconsistencies in the design, creation, and evaluation of composites can strongly affect the conclusions of composites studies and are the focus of this paper.

Numerous composite studies have been published within the field of solar-terrestrial physics, relating to a hypothesized connection between small changes in solar activity and the Earth’s atmosphere (e.g., [Tinsley et al. 1989](http://onlinelibrary.wiley.com/doi/10.1029/JD094iD12p14783/abstract); [Tinsley & Deen 1991](http://onlinelibrary.wiley.com/doi/10.1029/91JD02473/abstract); [Pudovkin & Veretenenko 1995](http://www.sciencedirect.com/science/article/pii/0021916994001092); [Stozhkov et al. 1995](http://link.springer.com/article/10.1007%2FBF02508564); [Egorova et al. 2000](http://www.sciencedirect.com/science/article/pii/S1364682600000808?via%3Dihub); [Fedulina & Laštovička 2001](http://www.sciencedirect.com/science/article/pii/S0273117701003039?via%3Dihub); [Todd & Kniveton 2001](http://onlinelibrary.wiley.com/doi/10.1029/2001JD000405/abstract), [2004](http://www.sciencedirect.com/science/article/pii/S1364682604001166); [Kniveton 2004](http://www.sciencedirect.com/science/article/pii/S1364682604001087); [Harrison & Stephenson 2006](http://rspa.royalsocietypublishing.org/content/462/2068/1221); [Bondur et al. 2008](http://link.springer.com/article/10.1134%2FS1028334X08070283); [Kristjánsson et al. 2008](http://www.atmos-chem-phys.net/8/7373/2008/acp-8-7373-2008.html); [Sloan & Wolfendale 2008](http://iopscience.iop.org/1748-9326/3/2/024001/); [Troshichev et al. 2008](http://www.sciencedirect.com/science/article/pii/S1364682608000886); [Svensmark et al. 2009](http://onlinelibrary.wiley.com/doi/10.1029/2009GL038429/abstract); [Dragić et al. 2011](http://www.astrophys-space-sci-trans.net/7/315/2011/astra-7-315-2011.html); [Harrison et al. 2011](http://rspa.royalsocietypublishing.org/content/467/2134/2777); [Laken & Čalogović 2011](http://onlinelibrary.wiley.com/doi/10.1029/2011GL049764/abstract); [Okike & Collier 2011](http://ieeexplore.ieee.org/xpl/articleDetails.jsp?arnumber=6051175); [Laken et al. 2012a](http://onlinelibrary.wiley.com/doi/10.1029/2012JD017683/full); [Mironova et al. 2012](http://www.atmos-chem-phys.net/12/769/2012/acp-12-769-2012.html); [Svensmark et al. 2012](http://www.atmos-chem-phys-discuss.net/12/3595/2012/acpd-12-3595-2012.html); [Artamonova & Veretenenko 2011](http://linkinghub.elsevier.com/retrieve/articleSelectSinglePerm?Redirect=http%3A%2F%2Fwww.sciencedirect.com%2Fscience%2Farticle%2Fpii%2FS1364682610001471%3Fvia%253Dihub&key=1ee359d22d707f4ad53c45d0f658924d9642b104); [Dragić et al. 2013](http://arxiv.org/abs/1304.7879)). However, despite extensive research in this area, clear conclusions regarding the validity of a solar-climate link from composite studies have yet to be drawn. Instead, the numerous composite analyses have produced widely conflicting results: some studies have shown positive statistical associations between the CR flux and cloud properties (e.g., [Tinsley & Deen 1991](http://onlinelibrary.wiley.com/doi/10.1029/91JD02473/abstract); [Pudovkin & Veretenenko 1995](http://www.sciencedirect.com/science/article/pii/0021916994001092); [Todd & Kniveton 2001](http://onlinelibrary.wiley.com/doi/10.1029/2001JD000405/abstract), [2004](http://www.sciencedirect.com/science/article/pii/S1364682604001166); [Kniveton 2004](http://www.sciencedirect.com/science/article/pii/S1364682604001087); [Harrison & Stephenson 2006](http://rspa.royalsocietypublishing.org/content/462/2068/1221); [Svensmark et al. 2009](http://onlinelibrary.wiley.com/doi/10.1029/2009GL038429/abstract), [2012](http://www.atmos-chem-phys-discuss.net/12/3595/2012/acpd-12-3595-2012.html); [Dragić et al. 2011](http://www.astrophys-space-sci-trans.net/7/315/2011/astra-7-315-2011.html),[2013](http://arxiv.org/abs/1304.7879); [Harrison et al. 2011](http://rspa.royalsocietypublishing.org/content/467/2134/2777); [Okike & Collier 2011](http://ieeexplore.ieee.org/xpl/articleDetails.jsp?arnumber=6051175)), while others find no clearly significant relationships (e.g., [Lam & Rodger 2002](http://www.sciencedirect.com/science/article/pii/S136468260100092X); [Kristjánsson et al. 2008](http://www.atmos-chem-phys.net/8/7373/2008/acp-8-7373-2008.html); [Sloan & Wolfendale 2008](http://iopscience.iop.org/1748-9326/3/2/024001/); [Laken et al. 2009](http://onlinelibrary.wiley.com/doi/10.1029/2009GL040961/abstract); [Laken & Čalogović 2011](http://onlinelibrary.wiley.com/doi/10.1029/2011GL049764/abstract); [Laken et al. 2011](http://onlinelibrary.wiley.com/doi/10.1029/2010JD014900/abstract); [Laken et al. 2012a](http://onlinelibrary.wiley.com/doi/10.1029/2012JD017683/full); [Čalogović et al. 2010](http://onlinelibrary.wiley.com/doi/10.1029/2009GL041327/full)), or even identify significant correlations of a negative sign (e.g., [Wang et al. 2006](https://inis.iaea.org/search/search.aspx?orig_q=RN:37116597); [Troshichev et al. 2008](http://www.sciencedirect.com/science/article/pii/S1364682608000886)). We suggest that these ambiguities may result from seemingly minor methodological differences between the composites (e.g., relating to the filtering or normalization of data), which are capable of producing widely divergent results. When the dataset is not suited to a particular method of statistical analysis, incorrect conclusions regarding the significance (*i.e.*, the probability $p$-value associated with composite means at given times) of the composites may be reached, which is part of the reason why these aforementioned studies have presented conflicting results.

In this paper, we aim to highlight the methodologies that may produce such conflicts. We provide details on how to properly perform composite analyses of geophysical datasets, and suggest a robust method for estimating the statistical significance of variations over the composite period. Although methods to assess the significance of variations over composites that account for non-random variations have been previously demonstrated (Forbush et al. 1982, 1983; Singh 2006), the incorrect application of statistical tests remains a common feature among composite analyses. Consequently, in this work we also present a further method of assessing statistical significance, which draws from randomized samples of the datasets themselves using a Monte Carlo (MC) methodology and, as a result, makes no assumptions regarding the properties of the data.

## 2. Working from the example of the hypothesized cosmic ray – cloud link ##

Throughout this work we will continue to use the example of testing the hypothesized CR flux/cloud connection to provide examples of various issues that may arise during composite analyses (albeit using real-world data rather than the idealized, fictitious data of Fig. 1). A link between the CR flux and cloud was initially suggested by [Ney (1959)](http://www.nature.com/nature/journal/v183/n4659/abs/183451a0.html), who theorized that the weather might be influenced by variations in the CR flux. [Dickinson (1975)](http://journals.ametsoc.org/doi/abs/10.1175/1520-0477%281975%29056%3C1240%3ASVATLA%3E2.0.CO%3B2) later proposed that such associations may result from an influence of atmospheric ionization produced by the CR flux on the microphysics of aerosols and clouds. Some of the first reports of positive correlations between clouds and cosmic radiation came from the composite studies (e.g., [Tinsley et al. 1989](http://onlinelibrary.wiley.com/doi/10.1029/JD094iD12p14783/abstract); [Pudovkin & Veretenenko 1995](http://www.sciencedirect.com/science/article/pii/0021916994001092); [Pudovkin et al. 1997](http://www.sciencedirect.com/science/article/pii/S0273117797007679)). These and subsequent studies selected time periods based on the occurrence of sudden, high-magnitude reductions in the CR flux impinging on the Earth’s atmosphere, termed Forbush decrease (Fd) events ([Forbush 1938](http://journals.aps.org/pr/abstract/10.1103/PhysRev.54.975)), generated by solar disturbances such as coronal mass ejections ([Lockwood 1971](http://link.springer.com/article/10.1007%2FBF00173346); [Cane 2000](http://link.springer.com/article/10.1023%2FA%3A1026532125747)).

As an example, we will work from a composite of 44 Fd events identified from the Mt. Washington Observatory from 1984 to 1995, located at 44.30°N, 288.70°E, 1,900 m above mean sea level, at a cut-off rigidity of 1.24 GV (the list of Fd events were obtained from http://www.ngdc.noaa.gov/stp/solar/cosmic.html). The Fd events were adjusted so that the maximum CR flux deviation associated with each Fd is aligned with the key composite date (*i.e.*, $t_{0}$ of the composite $x$-axis); without adjustment, the key date would instead be aligned to the date of Fd onset, which may differ from a period of hours-to-days from the maximal reduction in the CR flux (for further details see Laken et al. 2011). Fd events coincident within a ±7 day period of strong (>500 MeV) Solar Proton Events (SPEs) were excluded from the analysis; as such events may produce the opposite ionization effects to those we wish to isolate. Our CR flux data is the same as that of [Laken et al. (2012a)](http://onlinelibrary.wiley.com/doi/10.1029/2012JD017683/full), being a combination of daily averaged Climax Colorado and Moscow neutron monitor data centered on zero (Fig. 2a). In addition, we have also used global daily averaged International Satellite Cloud Climatology Project (ISCCP) D1 total cloud fraction (1000–50 mb) from IR-cloudy pixels (Fig. 2b) which covers the period from 01/07/1983 to 31/12/2009. Throughout this work we will frequently use the cloud data as an anomaly equivalent to $Ai$ described in **Section 1**, with units of %. We again note that the analysis presented here is not meant as a serious test for the hypothesized CR–cloud link, for which other similar studies exist (e.g., [Laken & Čalogović 2011](http://onlinelibrary.wiley.com/doi/10.1029/2011GL049764/abstract)), but rather is presented for demonstration purposes.

In [ ]:
data = pd.read_csv("Data/Data.csv")
date_fix = pd.read_csv("Data/dt_index.csv")      # Read original Date data
date_list = []
for entry in zip(date_fix['yyyy'],date_fix['mm'],date_fix['dd']):
    date_list.append(pd.datetime(*entry).date()) # Convert it to a datetime list
df = data.set_index(np.array(date_list))         # set the date list as the df index
cond1 = df != -999    # Check for missing data conditions and replace with NAN
df = df[cond1]

The dataframe contains the following from 1983-07-01 to 2009-12-31:

|Variable | Description |
| --- | --- |
|`cld`|  Daily globally avg. ISCCP D1 V13 data (IR-detected cloud cover) originally from NASAs EOSWEB site|
|`cld_360smth` |  the isccp variable with a 360-day box-car filter and edge (+-150day) truncation|
|`cld_360anm`| Daily values subtracted from the 360-day box-car filter|
|`cld_21smth`| a 21-day box-car filter on the cld dataset with edge truncation|
|`cld_21anom`| isccp subtracted from smth21 |
|`CRF`| cosmic ray flux data, derrived using moscow/climate neutron monitors from Laken et al. 2012|
|`CRF_21smth`| CRF data with 21-day box-car filter and edge truncation|
|`CRF_21anom`| CRF anomaly, calculated by subtracting CRF from CRF_21smth|

In [ ]:
TOOLS = "pan, wheel_zoom, box_zoom, reset"
p2a = figure(width=800, plot_height=300, tools=TOOLS, x_axis_type = "datetime")
p2a.line(df.index, df['CRF'], legend="Normalized cosmic ray flux (%)",color='black')

p2b = figure(width=800, plot_height=300,x_axis_type = "datetime", 
            title=None, x_range=p2a.x_range, tools=TOOLS)
p2b.line(df.index, df['cld'], legend='Cloud cover (%)', color='black')
p2b.line(df.index, df['cld_21smth'], legend="21-day smooth",color='red')

fig2 = gridplot([[p2a],[p2b]])
show(fig2)

**Figure 2** (a) Top panel, daily averaged normalized cosmic ray flux (%) calculated from Climax Colorado and Moscow neutron monitors from [Laken et al. (2012a)](http://onlinelibrary.wiley.com/doi/10.1029/2012JD017683/full). And (b) bottom panel, global, daily averaged, IR-detected cloud fractions (%) from the ISCCP D1 data.

## 3. Constructing a composite ##

### 3.1. Using composites to test a hypothesis ###

After successfully constructing a suitable composite for analysis, anomalies should be objectively examined for evidence of a CR flux-cloud connection via statistical methods. However, it is important to remember that statistics cannot prove any hypothesis; it can only provide a probability that a given hypothesis is or is not correct. Therefore, to test the existence of a hypothesized CR flux-cloud connection, we must construct a null hypothesis that may be tested and possibly rejected. In this instance, the null hypothesis ($H_{0}$) is that fluctuations observed over a composite of Fd events are indistinguishable from natural variability, while $H_{1}$, the alternate hypothesis, is that cloud variations distinguishable from normal (random) variability may be detected in association with the occurrence of Fd events. We must then select a confidence level at which to test our hypothesis: in this instance, we will present statistics for the commonly used 0.05 and 0.01 probability ($p$) value at the two-tailed confidence intervals (hereafter written as $p = 0.05$ and $p = 0.01$).

Detailed procedures relating to the statistical analysis of geophysical data in composite analyses have been published by Forbush et al. (1982, 1983) and Singh (2006), which demonstrate how to assess the significance of non-random (autocorrelated) data. Statistical significance has often been assessed in solar-terrestrial composite analyses by comparing composite means obtained at different times over a composite period, commonly periods prior to and following the occurrence of Fd events are utilized (e.g., [Pudovkin & Veretenenko 1995](http://www.sciencedirect.com/science/article/pii/0021916994001092); [Pudovkin et al. 1997](http://www.sciencedirect.com/science/article/pii/S0273117797007679); [Todd & Kniveton 2001](http://onlinelibrary.wiley.com/doi/10.1029/2001JD000405/abstract), 2004; Svensmark et al. 2009; Dragić et al. 2011; Okike & Collier 2011). In relation to this, we importantly note that although composites of geophysical data focusing on Fd events may be considered to be independent in the $n$-dimension (events), they are often highly autocorrelated (dependent) in the t-dimension (time). Thus, an analysis that seeks to compare composite means across $t$ (e.g., in the manner previously described) must account for autocorrelation effects.

In autocorrelated data the value at any given point in time is affected by preceding values. As a result, sequential data points are no longer independent, and so there is less sample variation within the dataset than there would otherwise be if the points were independent. If the variance and the standard deviation (σ) are calculated using the usual formulas, they will be smaller than the true values. Instead, the number of independent data points (effective sample size) must be used to scale the sample variance (Wilks 1997). This adjustment will produce confidence intervals that are larger than when autocorrelations are ignored, making it less likely that a fluctuation in the data will be interpreted as statistically significant at any given $p$-value. In composite analyses, the number of independent data points (*i.e.*, the effective length of the composite sequences) is equivalent to the effective sample size (Forbush et al. 1983): effective sample sizes may be calculated by the methods detailed in Ripley (2009) and Neal (1993). Despite the well-established nature of methods they are not widely applied within the solar-terrestrial community, with numerous studies implementing significance testing that simplistically assumes that the data are independent in the $t$-dimension.

In Section 3.4 we present an additional method of significance testing for composite studies based on Monte Carlo analysis approaches that is highly robust. This method does not rely on comparing composite means at given times over the composite to evaluate significance, but instead evaluates the p-value of each t-point individually, using probability density functions (PDFs) constructed from large numbers of randomly generated samples (with a unique PDF at each $t$-point).

### 3.2. Generating anomalies: the importance of static means in composite analyses ###

To effectively evaluate changes in daily timescale data, variations at timescales beyond the scope of the hypothesis testing should be removed from the dataset. Forbush et al. (1982, 1983) showed that bias can be introduced into traditional statistical tests if fluctuations at longer timescales are not removed. The removal of annual variations alone (which normally dominate geophysical data) will not remove fluctuations that cannot be eliminated by linear de-trending, such as those relating to the effects of planetary-to-synoptic scale systems. Such systems may extend over thousands of kilometers in area, and their influence on atmospheric variability may last from periods of days to several weeks; the random inclusion of their effects in composites may produce fluctuations at timescales shorter than seasonal considerations, yet greater than those concerning our hypothesis testing (so-called intermediate timescales) which may not be removed by linear de-trending. Consequently, when intermediate timescale fluctuations are not accounted for prior to an analysis, it may result in periods of shifted mean values and high autocorrelation being inadvertently included into composites. Suitable filters should be applied to the data to remove longer timescale variability, but to retain variations at timescales that concern the hypothesis testing. In the case of a CR flux-cloud hypothesis, a 21-day running mean (high-pass filter) may be suitable, as the microphysical effects of ionization on aerosol/cloud microphysics are expected to occur at timescales of < 1 week (Arnold 2006).

Although filtering data has the benefit of potentially reducing noise, it should be applied with caution, as it may also introduce artifacts, and reduce or even destroy potential signals altogether. Figure 3a–c shows the influence of a 21-day smooth filter on three idealized symmetrical disturbances of differing length. These disturbances are centered on t 0, each has an amplitude of 100 units in the y-axis dimension and span time periods of (a) 3-days, (b) 7-days, and (c) 11-days. To each time series, a smooth filter of 21-days (i.e., a box-car running mean) has been subtracted from the disturbance (the filters are shown as black lines). The resulting anomalies are shown as the red lines of each figure. As the duration of the idealized disturbance increases, so too does the appearance of overshoot artifacts. From Figure 3a–c, the original (peak-to-trough) signal of 0 to −100 has been shifted by overshoot effects by: (a) +9.6, (b) +19.0, and (c) +27.8. For a and b, the amplitude of the original disturbance is fully preserved, however, as the timescales of the disturbance increase further, the amplitude is reduced. E.g. for the 11-day disturbance the amplitude is diminished by 0.8% compared to the unfiltered signal, while for a 15-day disturbance (not shown) the amplitude decreases by 9%. The amount of signal attenuation will increase as the deviations approach the width of the smooth filter. For deviations at timescales of approximately 1/3rd the width of the smooth filter, the attenuation is negligible. Given this, filters should be applied with care to minimize noise while preserving potential signals. Following the application of the filter in this work we have constrained the reliability of our analysis to time periods of < 7 days.

In [ ]:
# Create a dataframe of idealised signals made up of disturbances, ie. a sharp reduction to -100 units,
# spanning a period of 3, 7, 11 and 15 days

def ideal_signal(dist = 7):
    """
    Return an idealised disturbance, peaking at -100 units, in a 1D np.array
    symetrically centered on index 21.
    dist = number of disturbed data points (default is 7)
    """
    signal = np.zeros(41)
    signal_max = -100.
    change_days = (int((dist -1) /2)) # points around key index [20] to inc/dec to signal max
    change_rate = signal_max / (change_days + 1)
    for n, day in enumerate(range(20 - change_days, 21)):
        signal[day] = (n + 1) * change_rate
    for n, day in enumerate(range(21, 21 + change_days)):
        signal[day] = signal[20] + (((n + 1) * change_rate) *-1.)
    return signal

ideal = pd.DataFrame(data=np.c_[ideal_signal(3), ideal_signal(7), ideal_signal(11), ideal_signal(15)],
                     columns=["3-day","7-day","11-day","15-day"], index=range(-20,21))

# Also give a list of values identified of cosmic ray flux change during averaged Forbush decreases
# where the fd is centered on index 40. 
# nb. 81 time points are included below for display due to the long tails of the rolling 21-day mean 
fd_real =np.array([-4.277,-4.452,-4.414,-4.309,-4.182,-4.345,-4.373,-4.596,-4.815,-4.781,-4.825,-4.975,
                   -5.221,-5.019,-5.132,-5.041,-4.875,-4.793,-4.761,-4.596,-4.899,-4.878,-4.877,-4.804,
                   -4.683,-4.875,-5.102,-5.152,-5.093,-4.909,-4.918,-5.054,-5.057,-4.823,-5.013,-5.001,
                   -5.044,-4.798,-5.022,-6.889,-8.972,-8.075,-7.671,-6.920,-6.844,-6.548,-6.510,-6.314,
                   -6.202,-6.048,-5.512,-6.177,-6.518,-6.583,-6.069,-5.755,-5.172,-5.159,-5.059,-5.239,
                   -5.274,-5.003,-4.794,-4.042,-4.794,-4.984,-4.810,-4.488,-4.592,-4.853,-5.089,-5.256,
                   -5.680,-5.545,-5.641,-5.514,-5.627,-5.749,-5.680,-5.579,-5.718])

In [ ]:
def get_21smth(values):
    """Return a np array of values for a 21-day running mean"""
    return np.nan_to_num(pd.rolling_mean(values, window=21, center=True))

p3a = figure(width=400, plot_height=300, title=None, tools=TOOLS)
p3b = figure(width=400, plot_height=300, title=None, x_range=p3a.x_range, y_range=p3a.y_range, tools=TOOLS)
p3c = figure(width=400, plot_height=300, title=None, x_range=p3a.x_range, y_range=p3a.y_range, tools=TOOLS)
for p_item, key in zip([p3a, p3b, p3c],["3-day","7-day","11-day"]):
    p_item.line(ideal.index, ideal[key],legend=key+' signal',color='blue')
    p_item.line(ideal.index, get_21smth(ideal[key]), legend="21 day avg",color='black')
    p_item.line(ideal.index, ideal[key] - get_21smth(ideal[key]), legend="anomaly",color='red')
    p_item.legend.location = "bottom_left"

p3d = figure(width=400, plot_height=300, title=None, x_range=p3a.x_range, tools=TOOLS)
p3d.line(range(-20,21),fd_real[20:61],legend='Fd signal',color='blue')
p3d.line(range(-20,21),get_21smth(fd_real)[20:61], legend="21 day avg", color='black')
p3d.line(range(-20,21), (fd_real[20:61] - get_21smth(fd_real)[20:61])+np.mean(fd_real[20:61]),
        legend="anomaly", color='red')
p3d.legend.location = "bottom_left"

fig3 = gridplot([[p3a, p3b],[p3c, p3d]])
show(fig3)

** Figure 3. ** (a–c) The differences between symmetrical idealized disturbances of increasing duration (3-day, 7-day, and 11-day) with an amplitude of 100 units (blue lines) centered on $t_{0}$, and the results of subtracting a 21-day smooth (box-car running mean) from the data which acts as a high-pass filter (black lines). The resulting filtered data (referred to as anomalies) are shown as red lines, and display overshoot effects. (D) Bottom right: the effect of subtracting the smooth filter from the CR flux (units in normalized counts). The anomaly has been offset by the Fd signal mean for plotting purposes.

Figure 3d shows a composite of $n = 44$ events, centered on the CR flux minimum of Fd events. CR flux data (blue line) are from the combined daily mean counts of the neutron monitors (NM) of Moscow and Climax, normalized by subtracting the mean of the 1950–2010 period (as in Fig. 2a). A 21-day smooth of these data are shown (black line). The normalized data show a peak-to-trough reduction between $t_{−3}$ and $t_{0}$ of 4.17% and a 21-day smooth (high-pass filter) of these data are also shown (black line). Following the removal of a 21-day high-pass filter from the normalized data, the resulting anomaly (red line) shows a slightly smaller peak-to-trough change over the same period of 4.05%. This indicates that 97.1% of the signal has been preserved following the application of the 21-day (high-pass) smooth filter. Overshoot distortion effects are also observable in the CR flux anomaly, and notably this produced an artificial enhancement of the increase in the CR flux prior to and following the Fd events.

Despite the limitations associated with data filtering, the noise reduction which may be gained by minimizing the influence of autocorrelated structures within data may be non-negligible. To exemplify the potential impacts of intermediate timescale structures on composites and the benefit of removing them, we have generated a random composite of cloud cover data with $n = 20$ events, over an analysis period of $t\pm40$ days. The $n = 20$ events upon which the composite was based were selected using a random number generator, ensuring that the chances of any one date (event) being selected for the composite were equal, and that once selected, a date could not re-occur in the composite. All of the random composites used throughout this work were created in this manner. The composite is presented in Figure 4a, with the cloud anomalies calculated by two methods. Firstly, through a simple removal of a linear trend from the cloud data (black line); this approach will remove linear bias from the composite resulting from seasonality. Secondly, we have also created an anomaly by subtracting a 21-day running mean (high-pass filter) from cloud cover (blue line). In this latter method, all variations (linear and non-linear) at timescales greater than 21-days are accounted for, including both seasonality and influences from intermediate timescale fluctuations. A period of 21-days was selected as this is three times greater than the upper limit response time predicted for a CR flux cloud relationship (Arnold, 2006), and so subtracting a running mean of this length from the data should not diminish the amplitude of any potential signals, yet it may still be useful in minimizing influences from longer term variations which are not the subject of the hypothesis testing. A comparison of these two approaches (Fig. 4a) gives an example of how the failure to remove non-linear intermediate timescale variations influences the composite, with noticeable deviations between the two methods. However, it is difficult to objectively assess the bias expressed as the difference between these two methods with only one random example. With this in mind histogram Figure 4b is presented, displaying the results of 100,000 randomly generated composites, showing the difference between the linear fit removed data and the 21-day smooth cloud anomaly at $t_{0}$. The histogram shows that the 1σ difference between these anomalies (*i.e.*, the impact of not accounting for intermediate timescale variations) is approximately 0.1%. We note this value will vary depending on the size of n and the length of the running mean applied, and some differences between the two filtering methods may also be attributed to overshoot errors. As we shall demonstrate in later sections, unaccounted-for biases of such amplitudes may have a non-negligible impact on the estimated statistical significance of composited anomalies.

### Composite functions ###

In [ ]:
def get_datelist(key_dates, period=40):
    """ Given a list of key dates (in datetime format) create
        return a dictionary of dates corresponding to the datetimes
        over a range of periods a specific range from the originals.
    """
    key_dics = {}
    for tdiff in range(period*-1, period+1): 
        tmp = []
        for date in key_dates:
            tmp.append(date + dt.timedelta(days=tdiff))
        key_dics[tdiff] = tmp
    return key_dics


#@profile(immediate=True)
def composite(time_series, date_dic, norm_prd=None):
    """
    Calculates composites from a dataset (time_series) when given a dictionary
    object containing lists of dates at lag (date_dic). The dates
    expected are as generated from get_datelist().
    Function returns arrays for the lag, composit means, and Standard Error of 
    the Mean uncertainty.
    The time_series object should be data from a pd Dataframe e.g. df.CRF_21anm.
    If a normalisation period is specified (as a list of integers relative to a
    key date) then normalization is performed,(e.g. composite(time_series=df.CRF,
    date_dic=d,norm_prd=[-10,-9,-8,-7,-6])).
    """
    xvals = sorted(date_dic.keys())
    k_size = len(date_dic[0])
    A = np.zeros(k_size)
    for key in xvals:
        A = np.vstack([A, time_series[date_dic[key]].values])
    A = A[1:,:]
    if norm_prd is None:
        return xvals, np.nanmean(A,axis=1), np.nanstd(A,axis=1)/np.sqrt(k_size)
    else:
        d_dayPosition = dict(zip(xvals,range(len(xvals))))
        subs = [d_dayPosition[n] for n in norm_prd]
        for event in range(len(date_dic[0])): 
            A[:,event] = A[:,event] - np.mean(A[subs, event])
        return xvals, np.nanmean(A,axis=1), np.nanstd(A,axis=1)/np.sqrt(k_size)

    
#@profile(immediate=True)
def montecarlo(dataset, its=1000, k=20, prd = 80, hide_status_bar = False, norm_prd=None):
    """
    Draws [its] random composite samples (of n=[k]) from a [dataset], over a 
    period of ±[prd]. Can also show its progress [hide_status_bar].
    --- Parameters ---
    dataset = time series data (e.g. df.cld)
    its = size of MC population
    k = number of dates (events) in the composites
    prd = number of days around key date to include
    progress = Boolean. Show the Monte Carlo progress.
    """
    mc = {}
    Sample = namedtuple('Random_result', ['y', 'y_sem'])
    for n in tqdm(range(its), disable=hide_status_bar):
        rnd_dates = random.sample(population=list(dataset.index), k=k)
        d = get_datelist(key_dates=rnd_dates, period=prd)
        x, y, y_sem = composite(date_dic=d, time_series=dataset, norm_prd=norm_prd) 
        tmp = Sample(y, y_sem)
        mc[n] = tmp
    mc['xdims'] = x
    return mc

def f4_compareFits(its=1000, k=20, prd=12 , hide_status_bar = False):
    """
    Generate the key-day diffrence between linear fits and 21-day smoothed data on
    day 0 of random composites. Returns a list of data for producing the histogram 
    of Figure 4b. Note, day 0 index is identified from the x list from the MC.
    --- Parameters ---
    its = size of MC population
    k = number of dates (events) in the composites
    hide_status_bar = Boolean. Show the Monte Carlo progress.
    """
    fit_diff = []
    mc = montecarlo(dataset=df.cld, its=its, k=k, prd=prd, hide_status_bar=hide_status_bar)
    x = np.array(mc.pop("xdims")) # Grab the x-dimensions from the dictionary and make it a np.array
    key_index = np.where(x == 0)[0][0]
    for key in mc:          # iterate over the dictionary keys (i.e. MC items)
        y = mc[key].y
        y_smooth = pd.rolling_mean(y, window=21, center=True)  # Calc. 21-day smooth and anom. (delta)
        y_anom = y - y_smooth
        a, b, r, p, stderr = linregress(x,y)                   # Use linear regression func. from scipy.stats
        yfit = [xval * a + b for xval in x]                    # Calc. a fit with slope (a) and intercept (b)  
        linear_anom = y - yfit                                 # Calc. anom. (delta) from the linear fit 
        fit_diff.append(y_anom[key_index] - linear_anom[key_index])
    return fit_diff

def summarise_distribution(dist):
    """Print some summary stats of a distribution of values"""
    print("Mean of MC distibution: μ = {0:3.2f}".format(np.mean(dist)))
    print("Center of MC distibution: 50th percentile = {0:3.2f}".format(np.median(dist)))
    print("Spread of MC distibution: 1σ = {0:3.2f}".format(np.std(dist)))
    return

def gen_pdf(x_val, sigma, mu):
    """ Return a normalised Gaussian PDF for each x-axis point"""
    y_pdf = 1/(sigma * np.sqrt(2 * np.pi)) * np.exp(-(x_val - mu)**2 / (2 * sigma**2))
    return y_pdf / np.sum(y_pdf)

In [ ]:
fit_diffs = f4_compareFits(its=5000, hide_status_bar=False)

In [ ]:
hist, edges = np.histogram(fit_diffs, density=False, bins=100)
norm_hist = hist/np.sum(hist) # Normalise the histogram so unity = 1.
summarise_distribution(norm_hist)

# Exact random set of 20 dates used as an example in the paper:
fig4a_rnd = [661, 1809, 2005, 2259, 2559, 3572, 4139, 5328, 5488, 5608,
             5685, 5883, 6445, 7399, 7525, 7822, 7942, 8205, 8570, 9053]
fig4a_dates = [date_list[indx] for indx in fig4a_rnd]
d = get_datelist(key_dates=fig4a_dates, period=80)
x, y, yerr = composite(date_dic=d, time_series=df.cld) # Composite the cloud data using the dic dates
y_smooth = pd.rolling_mean(y, window=21, center=True)  # 21-day smooth and anomaly
y_anom = y - y_smooth
a, b, r, p, stderr = linregress(x,y) # Linear regression and anomaly
yfit = [xval * a + b for xval in x] 
linear_anom = y - yfit

In [ ]:
# Figure 4 plotting
p4a = figure(width=400, plot_height=400, title=None, tools=TOOLS)
p4a.line(x[40:121], y_anom[40:121],legend='from moving avg.',color='blue')
p4a.line(x[40:121], linear_anom[40:121],legend="from linear fit", color='black')
p4a.legend.location = "top_left"
p4a.xaxis.axis_label = "Days since key date"
p4a.xaxis.axis_label_text_font_size = '10'
p4a.yaxis.axis_label = "δ cloud cover (%)"
p4a.yaxis.axis_label_text_font_size = '10'


p4b = figure(width=400, plot_height=400, title=None, tools=TOOLS)
p4b.quad(top=norm_hist, bottom=0, left=edges[:-1], right=edges[1:],
     fill_color="#ff9900", line_color="#ff9900", legend="MC population")
xvals_pdf = np.linspace(-0.1,0.1,num=100, endpoint=True)
p4b.line(xvals_pdf, gen_pdf(x_val=xvals_pdf, mu=np.average(fit_diffs), sigma=np.std(fit_diffs)),
         legend='PDF',color='black', line_width=1.5)
p4b.legend.location = "top_left"
p4b.xaxis.axis_label = "Diff. b/w two methods to calc. anomalies (at t₀)"
p4b.xaxis.axis_label_text_font_size = '10'
p4b.yaxis.axis_label = "Probability density"
p4b.yaxis.axis_label_text_font_size = '10'

fig4 = gridplot([[p4a, p4b]])
show(fig4)
del fit_diffs # free the memory

**Figure 4** (a) Left panel: one random instance of $n = 20$ events, for both linearly detrended data (black line) and an anomaly from a 21-day moving average (blue line). Differences between these two curves show the possible influences of intermediate timescales on the data. (b) Right panel: the differences between these curves at $t_{0}$ for 10,000 random instances (of $n = 20$). The histogram shows a $1\sigma$ value of 0.1%, a non-trivial difference which may influence the outcome of a composite analysis.

* *nb. The text above reflects the original paper, but from the re-code it seems 1σ = 0.01, much less variance. Not sure yet where the larger variance came from in the original, but may potentially be a bug.*

### 3.3. Using signal-to-noise-ratio as a constraint when designing a composite ### 

The data we have presented in Figure 2b are global in extent and utilize all pressure levels of the ISCCP dataset (1000–50 mb). However, composite studies often use subsets of data for analysis, with varying spatio-temporal restrictions: e.g., focusing on stratospheric layers over Antarctica (Todd & Kniveton 2001, 2004; Laken & Kniveton 2011); areas over isolated Southern Hemisphere oceanic environments at various tropospheric levels (Kristjánsson et al. 2008); low clouds over subtropicaltropical oceans (Svensmark et al. 2009); indicators of the relative intensity of pressure systems (vorticty area index) over regions of the Northern Hemisphere Mid-latitude zone, during differing seasons (Tinsley et al. 1989; Tinsley & Deen 1991); total ozone at latitudes between 40°N and 50°N, during periods of high solar activity and eastern phases of the quasi-biennial oscillation (Fedulina & Laštovička 2001); variations in atmospheric pressure and formation of cyclones/anticyclones over the North Atlantic (Artamonova & Veretenenko 2011); and diurnal temperature ranges over areas of Europe (Dragić et al. 2011; Dragić et al. 2013; Laken et al. 2012a). In addition to spatial/seasonal restrictions, it has been frequently proposed that perhaps a CR-cloud effect is only readily detectable during the strongest Fd events (e.g., Harrison & Stephenson 2006; Svensmark et al. 2009, 2012; Dragić et al. 2011). Although such propositions are rational, high-magnitude Fd events are quite rare. For example, of the 55 original Fd events between 1984 and 1995 listed in the NOAA resource (discussed in Sect. 2), the Fd intensity (I Fd, the daily absolute percentage change of background cosmic ray intensity as observed by the Mt. Washington neutron monitor) is as follows: 28 weak events (I Fd < 5%), 21 moderate events (5% ≤ I Fd <$10$%), and 6 strong events (I Fd ≥ 10%). Consequently, composites focusing only on intense Fd events are invariably small in size, and therefore highly restricted by noise (Laken et al. 2009; Čalogović et al. 2010; Harrison & Ambaum 2010).

While there is often a good theoretical basis for restricting the sampling area, such constraints considerably alter the potential detectability of a signal. Restricting either the spatial area (hereafter a) of the sampling region or the number of sample events (composite size, hereafter n) will increase the background variability (noise) in composites. This relationship is quantitatively demonstrated in Figure 5, which shows how noise levels in random composites constructed from cloud cover anomalies (21-day running mean subtracted) vary for differing area a and composite size n. Using MC simulations this is calculated for composites varying over an a of 0.1–100% of the global surface, and $n$ of 10–1,000. For each combination of variables a and n a probability density function (PDF) of 10,000 randomly generated composite means at $t_{0}$ are generated and the 97.5th percentile of the distribution is then presented in Figure 5. For each corresponding a and $n$, any potential CR flux-induced cloud variations (which we shall refer to as a signal) must overcome the values shown in Figure 5 in order to be reliably detected; so these values represent the upper confidence level. A nearly symmetrical 2.5 percentile lower confidence level (not shown) is also calculated for every $a$ and $n$ combination, and together these upper and lower confidence intervals constitute the $p = 0.05$ significance threshold.

#### Use Plotly (offline mode) to reproduce an enhanced Figure 5 from pre-calculated data ####

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import Layout, Scene, Margin, Figure, Surface, YAxis, ZAxis, XAxis, ColorBar
from plotly import __version__
print("Using Plotly version {0}".format(__version__)) # Requires version >= 1.9
init_notebook_mode()

In [ ]:
# Load the z-dimensional data from numpy binary files
z975 = np.load('Data/surface_p975.npy')
z50 = np.load('Data/surface_p50.npy')
z025 = np.load('Data/surface_p025.npy')
# Set the axis labels for y and x dimensions
yind = [0.1,0.2,0.3,0.6,1.2,2.4,3.8,6.1,9.1,12.1,20.9,34.8,48.7,62.6,83.4,100.0]
xind = [10,20,30,40,50,60,70,80,90,100,150,200,400,600,1000]

In [ ]:
#How the upper/lower Percentile Rank (PR), indicating confidence intervals, vary with a|n
upper_scl = [[0,"#ffc266"],[1, "#e60000"]]# col of min level (from 'zmin') and max level (from 'zmax')
mid_scl = [[0,"#999999"],[1, "#737373"]]
lower_scl = [[0,"#0000e6"],[1, "#ccebff"]]

data = [
    dict(z=z975, type='surface',y=yind, x=xind, hoverinfo='all', showscale=False,
         name='97.5th PR', zauto=False, zmin=0, zmax=10, colorscale=upper_scl),
    dict(z=z50, y=yind, x=xind, opacity=0.95, type='surface',
         name='50th PR', zauto=False, zmin=-0.1, zmax=0.1, colorscale=mid_scl, showscale=False),
    dict(z=z025, y=yind, x=xind, opacity=1.0, type='surface',
         hoverinfo='all',name='2.5th PR', zauto=False, zmin=-10, zmax=0, colorscale=lower_scl,
         showscale=False,)]
         
axisatt_z = dict(autorange=True, showticklabels=True, title='δ')
axisatt_y = dict(type='log', autorange=True, showticklabels=True, title='a')
axisatt_x = dict(type='log', autorange=True, showticklabels=True, title='n')

## Layout info here
layout = dict(margin=Margin(l=0,r=0,t=0, b=0),
              scene=Scene(xaxis=XAxis(axisatt_x), yaxis=YAxis(axisatt_y), zaxis=ZAxis(axisatt_z),
                         cameraposition=[[0.45, 0.3, -1.0, 0.45], [0, 0, 0], 1.8]))
#Cameraposition: first 4 numbers are rotation, the next 3 are x,y,z translation, and the final number is zoom
fig = dict(data=data, layout=layout)
iplot(fig, show_link=False)

**Figure 5** The relationship between increasing sample area (as a percentage of global area: $x$-axis, denoted by a) and number of events in a composite (z-axis, denoted by n) on noise levels indicated by the 97.5 percentile mean cloud fraction anomalies (%, $y$-axis) within composites. These values constitute the upper $p = 0.05$ confidence interval. Each data point is calculated from 10,000 Monte Carlo (MC) simulations from cloud cover anomaly data (21-day running mean subtracted) using PDFs. All axes shown are logarithmic.

*nb.* To reproduce this work you will need to download the gridded ISCCP D1 IR Cloud fraction data set (avaiable from [NASA's EOSweb](https://eosweb.larc.nasa.gov/sse/)), and composites varying the $n$ and $a$ dimensions. The surface plot shows the Percentile Rank (PR) values for each $n$ $a$ combination from Monte Carlo distributions at the 97.5th, 50th, and 2.5th percentiles (indicating statistical confidence intervals). This figure is an enhanced version of the [original](http://www.files.benlaken.com/documents/Laken_Calogovic_2013/swsc130020-fig5.pdf).

The noise associated with composites of differing a and n sizes defines the lower-limit relationship which can be detected at a specified confidence level. Thus, for a specific magnitude of CR flux reduction, the minimum necessary efficiency at which a CR-cloud relationship must operate in order to be detected at a specified $p$-value can be calculated. For example, if composites were constructed with $n = 10$ samples from data with an area of only 1% of the Earth’s surface, then a change in cloud cover would have to exceed approximately $\pm6.3%$ in order to be classified as statistically significant at the $p = 0.05$ two-tailed level.

Based on such calculations, composites can be constructed from which a response can realistically be identified. For example, the most favorable study of a CR-cloud link by Svensmark et al. (2009) finds using a sample of $n = 5$ Fd events that following an 18% reduction in the CR flux a 1.7% decrease in cloud cover occurs. Regardless of the criticisms of this study (given in Laken et al. 2009 and Čalogović et al. 2010), we use this observation to calculate the most favorable upper limit (UL) sensitivity of cloud cover changes to the CR flux (1.7/18 giving a factor of 0.094). From a consideration of this UL sensitivity and the relationship between composite noise to area a and composite size n in Figure 5, we may then surmise that a CR flux-induced change in cloud cover (a signal), could almost certainly never be distinguishable from background variability (noise) at the $p = 0.05$ level, from a composite of 10 Fd events with an average CR flux reduction of 3% (the value of 3% is typically used to define a Fd event, Pudovkin & Veretenenko 1995; Todd & Kniveton 2001). This is because the UL sensitivity suggests that the CR flux would at most produce a cloud signal of 0.28% (0.094 × 3) assuming a linear CR-cloud relationship. Even if 100% of the globe were considered in the analysis the sample noise would still be too great (by a factor of 2) to detect the signal. For composite size of n = 10 Fd events we see the upper p = 0.05 confidence interval never drops below 0.6% due to the high noise levels of the data (Fig. 5). To significantly identify a signal with an amplitude of 0.28% in the presented cloud cover data above the $p = 0.05$ level, considering 100% of the globe, would require a composite size of at least $n = 50$.

### 3.4. Estimating the significance of anomalies by Monte Carlo methods ###

Evaluating the statistical significance of composited geophysical data by a comparison to randomly obtained samples is not a novel idea. One of the first applications of this method within the field of solar-terrestrial physics appears in (Schuurmans & Oort 1969) (hereafter SO69), who investigated the impacts of solar flares on Northern Hemisphere tropospheric pressures over a composite of 81 flare events. They constructed their composite over a three-dimensional (longitude, latitude, height) grid and examined changes over a 48 h period. They then evaluated the significance of their observed pressure variations at the $p = 0.05$ significance level (calculated from the standard deviation of their data). SO69 noted that due to autocorrelation in their data, the number of significant grid points beyond the $p = 0.05$ level may exceed the false discovery rate without there necessarily being a significant solar-related signal in their data (a point which we shall discuss further later in this work).

To assess the potential impact of autocorrelation on their data and more accurately gauge how unusual their result was, SO69 constructed three random composites of equal size to their original composite ($n = 81$), and compared the random results to their original findings. Although SO69 were limited by the lack of computing power available to them, and consequently their number of random composites was not sufficient to precisely identify the statistical significance of their flare composite, our method, which we will shortly describe in detail, is based upon the same principles as those of SO69. In essence, we will use populations of randomly generated composites and identify threshold significance values using probability density functions (PDFs) of these values, from which we may precisely evaluate the statistical significance of variations in the composite mean over $t$.

To achieve this, we can use the MC-based method of estimating noise used to calculate Figure 5 to provide a simple, yet powerful method of evaluating the statistical significance of composite means. From MC-generated PDFs we may test how likely it is that we can randomly obtain composite means of a given magnitude by chance. Events are selected at random from the data being scrutinized, and composites of equal sample sizes to the original composite are constructed. The data of the MC must be treated in exactly the same manner as that of the composite being evaluated with respect to the calculation of anomalies, or application of any normalization procedures (i.e., if the composite is based on an anomaly calculated from a 21-day running mean, then the MC-generated composites must also be based on data with this treatment). Following this, the composite mean at each t is calculated. If enough available data exist, this procedure can be repeated many times, making it possible to build up large PDFs of composite means for each t. From these, confidence intervals at specific p-values may then be identified as percentiles of the calculated distributions. We note that this procedure may fail in cases where insufficient data exists to build up distributions which are representative of the parent population from which the samples are drawn.

Geophysical data do not often follow idealized Gaussian distributions, as a result the two-tailed confidence intervals should be identified asymmetrically for optimum precision: i.e., the upper/lower $p = 0.05$ confidence interval should correspond to the 2.5th and 97.5th percentiles of the cumulative frequency, not simply to the $\pm1.96\sigma$ value. This is demonstrated in Figure 6, which shows a distribution of both the CR flux anomaly and cloud anomaly data, comprised of all data points from 1983–2010 (in this instance the data presented are the entire population of daily values rather than composite values). The two-tailed $p = 0.05$ thresholds are calculated from both percentiles (blue lines) and from the $\pm1.96\sigma$ level around the mean (red dashed lines). Under an ideal Gaussian distribution (displayed on the black lines) the 2.5/97.5th percentile and $\pm1.96\sigma$ values would be both equivalent and symmetrical around the mean value. However, due to the departures from the ideal Gaussian distribution there are differences between these significance values: this is most prominent in the upper confidence interval of the CR flux data where the deviation between the $+1.96\sigma$ and 97.5th percentile values is $0.16$% (Fig. 6a).

### Code for Figure 6 ###

In [ ]:
def return_good_values(dat):
    """Removes the nan values from series and converts to numpy array"""
    tmp = []
    for n in dat:
        if n == n:
            tmp.append(n)
    return np.array(tmp)    


def return_hist_data(dat, bins=100):
    """
    Takes df values. Returns the objects required to plot the histogram and fit.
    """
    tmp = return_good_values(dat)
    hist, edges = np.histogram(tmp, density=False, bins=bins)
    norm_hist =hist / np.sum(hist)
    xvals_pdf = np.linspace(min(edges),max(edges),num=len(edges)-1, endpoint=True)
    pdf = gen_pdf(x_val=xvals_pdf, mu=np.average(tmp), sigma=np.std(tmp))
    return norm_hist, edges, xvals_pdf, pdf


def returnSigPlotLines(dat):
    """Return a list of Bokeh Span objects for plotting vertical lines in figure 6."""
    tmp = return_good_values(dat)
    upper_sigma = np.median(tmp) + (np.std(tmp) * 1.96)
    lower_sigma = np.median(tmp) - (np.std(tmp) * 1.96)
    upper_percentile = np.percentile(tmp, 97.5)
    lower_percentile = np.percentile(tmp, 2.5)
    line1 = Span(location=upper_percentile, dimension='height', 
                 line_color='blue', line_width=1.0)
    line2 = Span(location=upper_sigma, dimension='height', line_color='red',
                 line_dash='dashed', line_width=1.0)
    line3 = Span(location=lower_percentile, dimension='height', line_color='blue',
                 line_width=1.0)
    line4 = Span(location=lower_sigma, dimension='height', line_color='red',
                 line_dash='dashed', line_width=1.0)
    return [line1, line2, line3, line4]

In [ ]:
# Figure 5 plotting
cr_norm_hist, cr_edges, cr_xvals_pdf, cr_pdf = return_hist_data(df.CRF_21anm, bins=500)

p5a = figure(width=400, plot_height=400, title=None, tools=TOOLS)
p5a.quad(top=cr_norm_hist, bottom=0, left=cr_edges[:-1], right=cr_edges[1:],
     fill_color="#ff9900", line_color="#ff9900")
p5a.line(cr_xvals_pdf, cr_pdf, color='black', line_width=1.0)
p5a.xaxis.axis_label = "CR flux anomaly (%)"
p5a.xaxis.axis_label_text_font_size = '10'
p5a.yaxis.axis_label = "Population density"
p5a.yaxis.axis_label_text_font_size = '10'
p5a.renderers.extend(returnSigPlotLines(df.CRF_21anm))


cld_norm_hist, cld_edges, cld_xvals_pdf, cld_pdf = return_hist_data(df.cld_anm21, bins = 500)

p5b = figure(width=400, plot_height=400, title=None, tools=TOOLS)
p5b.quad(top=cld_norm_hist, bottom=0, left=cld_edges[:-1], right=cld_edges[1:],
     fill_color="#ff9900", line_color="#ff9900")
p5b.line(cld_xvals_pdf, cld_pdf, color='black', line_width=1.0)
p5b.xaxis.axis_label = "Cloud anomaly (%)"
p5b.xaxis.axis_label_text_font_size = '10'
p5b.yaxis.axis_label = "Population density"
p5b.yaxis.axis_label_text_font_size = '10'
p5b.renderers.extend(returnSigPlotLines(df.cld_anm21))

fig5 = gridplot([[p5a, p5b]])
show(fig5)

** Figure 6 ** Distribution of daily anomalies for the entire 1983–2010 data period for: (a) cosmic ray flux (%), and; (b) cloud cover (%) data. For comparison an idealized Gaussian distribution is shown on the black lines. The mean ±1.96σ, and 2.5/97.5th percentile values are displayed on the red dashed and blue solid lines respectively.

Although the deviations between the $\pm1.96\sigma$ and 97.5th percentiles presented here are relatively small, these differences can become highly exaggerated depending on the distribution of the data under investigation, and so this should be taken into proper consideration. We also stress that when using this technique to determine significance thresholds of composites, the confidence intervals should be calculated for each time step ($t$) in the composite individually. Similar solutions across the t-dimension will only be produced by MCs if the analyzed data possess an equal variance and static means.

When using MC methods there is a question as to how many simulations are sufficient to obtain a reliable result; if too few are used a reliable solution will not be identified (i.e., running the MC numerous times with an identical set-up will produce a wide range of values). Conversely, setting the number of simulations too high may be inconvenient as it may require considerable computation time. As the number of simulations increases in a MC it becomes increasingly more likely that by repeating the MC you would arrive at a consistent result (this is termed convergence). In general, the amount of convergence achieved by the MCs increases exponentially with increasing number of simulations. To exemplify this, we have calculated results for 100 different MCs run with identical inputs for a range of simulation sizes between 5 and 200,000. Each MC generated random composites of $n = 40$ events from cloud anomaly data, and calculated a distribution of the means at each simulation size. The $\sigma$ of the 100 means for each simulation size is plotted in Figure 7, and indeed shows an exponential reduction (convergence) with increasing simulation size. Following the 80:20 rule (also known as the Pareto principle), we find that 80% of the convergence in the case of Figure 7 occurs after only a relatively small number of simulations (only 22); the remaining precision is achieved at an exponentially decreasing rate. However, we note that as computing these values requires only relatively limited resources in this instance, it was possible for us to use large (10,000 simulations) to achieve a near-fully converged result throughout the remainder of this work.

In [ ]:
# Get data from file holding pre-calculated convergence data (I will add ability to re-caclulate too...)
mc_convergence = pd.read_csv("Data/mc_convergence.txt",index_col=0)

a, b, r, p, err = linregress(np.log(mc_convergence.index), np.log(mc_convergence.sigma.values))
yvals = np.log(np.arange(1,20000))
yfit = yvals * a + b

In [ ]:
# Find the Pareto (80:20) point
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

ymax = max(np.exp(yfit))
ymin = min(np.exp(yfit))
yrange = ymax - ymin
pareto_y = ymax - (yrange * 0.8)
close = find_nearest(array=np.exp(yfit), value = pareto_y)
pareto_x = int(np.exp(yvals)[np.where(np.exp(yfit) == close)[0][0]])

In [ ]:
TOOLS = "pan, wheel_zoom, box_zoom, hover, reset, resize"
fig7 = figure(width=600, plot_height=400, title=None, tools=TOOLS, x_axis_type="log")

fig7.circle(mc_convergence.index, mc_convergence.sigma,legend="MC simulation",color='blue')
fig7.line(np.exp(yvals), np.exp(yfit), legend="𝑦=0.182𝑒 ^-0.517𝑥",color='black')

fig7.xaxis.axis_label = "σ between 100 composites (%)"
fig7.xaxis.axis_label_text_font_size = '10'
fig7.yaxis.axis_label = "No. simulations"
fig7.yaxis.axis_label_text_font_size = '10'

line1 = Span(location=pareto_x, dimension='height', line_color='red', line_width=0.75)
line2 = Span(location=pareto_y, dimension='width', line_color='red', line_width=0.75)
fig7.renderers.extend([line1,line2])

show(fig7)

**Figure 7** Convergence of 100 different Monte Carlo (MC) simulations on a consistent solution, indicated by an exponential reduction in the standard deviation (σ) between the 100 means calculated for random composites of $n = 40$ events. The number of simulations within each MC increases along the $x$-axis, from 5 to 200,000 (calculated values are indicated by diamond markers). An exponential fit to these data are plotted (solid line), along with the value of the Pareto principle (i.e., the $80:20$ rule), calculated from the fit, which indicates how many simulations are required for 80% of convergence to occur denoted by the intersect of the red lines.

To re-state and emphasize this procedure, in this work 10,000 MC simulations are used to generate PDFs of composite means expected at each instance of $t$, against which the significance of a composite may be evaluated. In each instance the mean of the PDFs should be zero as the 21-day filtered data has a mean close to zero (as indicated in Fig. 7), and the distribution of the PDF around zero will reflect the remaining (short-term) variability in the data. The PDFs show the variations expected from composites of a specified n-size in the absence of deterministic effects (i.e., when there are only random fluctuations occurring), and they therefore provide a basis from which to accept or reject the $H_{0}$ and $H_{1}$ hypothesis.

### 3.5. Applying the MC significance testing to real data ###

We present an example of the MC-based significance testing methodology applied to composites of Fd events for daily mean anomalies of CR flux (Fig. 8a) and cloud data (Fig. 8b). The composite of Figure 8 uses $n = 44$ adjusted Fd events, and is presented over a period of $t\pm40$. Only Fd events not coincident within a $t\pm7$ day period of ground level enhancement (GLE) events have been included in the composite (as described in Sect. 2). The mean of these data along with the $\pm1.96$ SEM values are shown on the solid black and dashed blue lines respectively. To these data, we have applied the previously discussed MC-based method of calculating confidence intervals as a test of statistical significance. These confidence intervals are calculated from PDFs of 10,000 MC simulations at each $t$-point, and are plotted for the $p = 0.05$ and $p = 0.01$ two-tailed confidence intervals (dashed and dotted red lines, respectively). The small variations in the individual confidence intervals of Figure 8 which can be observed across $t$ indicate the amount of convergence remaining to be achieved by the MCs.

In [ ]:
# Get integers from a file indiciating the date
tmp = []
with open("Data/FD_events.txt") as f:
    for line in f:
        tmp.append(int(line.split("\n")[0]))

# Make a list of date-time objects forthe key dates of the composite
fd_events = []
for day in tmp:
    fd_events.append(date_list[day])
    
# Create a dictionary of days ± period around the key dates
d = get_datelist(key_dates=fd_events, period=40)

# Make the composites
x, y_crf, yerr_crf = composite(date_dic=d, time_series=df.CRF_21anm)
x, y_cld, yerr_cld = composite(date_dic=d, time_series=df.cld_anm21)

In [ ]:
# This is pretty slow, look at speeding up with Numba
def mc_confidence_intervals(dataset, its, k, prd, norm_prd=None, hide_status_bar=False):
    """
    Use the Montecarlo function, restructure the output to be a 2D numpy array.
    Where the cols are composite days (t), and rows are individual MC simulations.
    Scan the columns (t-dimension) using the np.percentile function to identify
    confidence intervals. Return a dictionary of the confidence intervals at each t-step.
    """
    mc = montecarlo(dataset=dataset, its=its, k = k, prd = prd, norm_prd=norm_prd,
                    hide_status_bar = hide_status_bar)
    xvals = sorted(mc.pop("xdims"))
    A = np.zeros(len(xvals))
    for key in sorted(mc.keys()):
        A = np.vstack([A, mc[key].y])
    A = A[1:,:]
    p975 = []
    p995 = []  # Should change this boiler plate code...
    p025 = []
    p005 = []
    for n, day in enumerate(xvals):
        p975.append(np.percentile(A[:,n], 97.5))
        p025.append(np.percentile(A[:,n], 2.5))
        p995.append(np.percentile(A[:,n], 99.5))
        p005.append(np.percentile(A[:,n], 0.5))
    conf = {}
    conf['xvals']=np.array(xvals)
    conf['p975']=np.array(p975)
    conf['p995']=np.array(p995)
    conf['p025']=np.array(p025)
    conf['p005']=np.array(p005)
    if norm_prd is not None:  # If you want to extract the std over the normalisation
        d_dayPosition = dict(zip(xvals,range(len(xvals)))) # prd,(needed in Fig 9)
        subs = [d_dayPosition[n] for n in norm_prd]
        conf['norm_sigma']= np.std(A[:, subs])
    return conf

In [ ]:
%%time
ci_crf = mc_confidence_intervals(dataset=df.CRF_21anm, k=44, its=5000, prd=40)

In [ ]:
%%time
ci_cld = mc_confidence_intervals(dataset=df.cld_anm21, k=44, its=5000, prd=40)

### Plot figure 8 ###

In [ ]:
def make_patch(x, y, yerr):
    """Return paths for a patch object, based on the Bokeh bollinger example 
    (https://github.com/bokeh/bokeh/blob/master/examples/plotting/file/bollinger.py)
    """
    band_x = np.append(x, x[::-1])
    band_y = np.append(y + yerr, y[::-1] - yerr[::-1]) 
    return band_x, band_y

In [ ]:
p8a = figure(width=450, plot_height=400, title=None, tools=TOOLS)
p8b = figure(width=450, plot_height=400, title=None, x_range=p8a.x_range, tools=TOOLS)

crf_x, crf_y = make_patch(x=x, y=y_crf, yerr=yerr_crf)

p8a.patch(crf_x, crf_y, color='#9999ff', fill_alpha=0.5, line_alpha=0.5)
p8a.line(x, y_crf, color='black')
p8a.line(ci_crf['xvals'], ci_crf['p975'],color='red', line_width = 1.0, line_dash=[3])
p8a.line(ci_crf['xvals'], ci_crf['p995'],color='red', line_width = 1.0, line_dash = [1])
p8a.line(ci_crf['xvals'], ci_crf['p025'],color='red', line_width = 1.0, line_dash=[3])
p8a.line(ci_crf['xvals'], ci_crf['p005'],color='red', line_width = 1.0, line_dash=[1])

cld_x, cld_y = make_patch(x=x, y=y_cld, yerr=yerr_cld)

p8b.patch(cld_x, cld_y, color='#9999ff', fill_alpha=0.5, line_alpha=0.5)
p8b.line(x, y_cld, color='black')
p8b.line(ci_cld['xvals'], ci_cld['p975'],color='red', line_width = 1.0, line_dash=[3])
p8b.line(ci_cld['xvals'], ci_cld['p995'],color='red', line_width = 1.0, line_dash=[1])
p8b.line(ci_cld['xvals'], ci_cld['p025'],color='red', line_width = 1.0, line_dash=[3])
p8b.line(ci_cld['xvals'], ci_cld['p005'],color='red', line_width = 1.0, line_dash=[1])

p8a.yaxis.axis_label = "σ cosmic ray flux (%)"
p8a.yaxis.axis_label_text_font_size = '10'
p8a.xaxis.axis_label = "Days since FD event"
p8a.xaxis.axis_label_text_font_size = '10'

p8b.yaxis.axis_label = "σ cloud cover (%)"
p8b.yaxis.axis_label_text_font_size = '10'
p8b.xaxis.axis_label = "Days since FD event"
p8b.xaxis.axis_label_text_font_size = '10'

fig8 = gridplot([[p8a, p8b]])
show(fig8)

** Figure 8** Composites of ($n = 44$) Forbush decrease events over a $t \pm40$ day period not coincident within a $t \pm7$ day period of GLEs from 1983 to 1995 for: a) the CR flux (%), and b) cloud cover (%). The mean anomalies (solid black lines) are plotted together with the $\pm1.96$ SEM values (blue dashed lines) for each of the $t \pm40$ days of the composite. Based on PDFs of 10,000 MC simulations at each $t$ value two-tailed confidence intervals are calculated at the $p = 0.05$ and $p = 0.01$ levels (dashed and dotted red lines respectively).

*nb. In this version I have used percentiles instead of $\sigma$ levels.*

Daily mean CR flux anomalies and the ±1.96 SEM values are presented in Figure 8a: at $t_{0}$, CR flux anomalies of $−3.01 \pm 0.53\%$ are observed, corresponding to deviations of $18.0 \pm 3.2\sigma$ (the associated statistical significance of the mean and SEM ranges are all $p < 0.00$). Additionally, highly significant positive CR flux anomalies occur both before and after $t_{0}$, the largest of which being during $t_{−3}$, where a CR flux anomaly of $0.89 \pm 0.37\%$ was observed ($6.3 \pm 2.1\sigma$, again with the mean and SEM ranges all significant at $p \lt 0.00$). Increases of CR flux prior to the Fd correspond to the influence of a shock/sheath structure of coronal mass ejections (CME) that generate the Fd event, where the CRs are swept up and deflected by the propagating magnetic disturbance generated by a CME. Increases of CR flux after Fd events can be connected with overrecovery effects in some cases (Dumbović et al. 2012). However, in our case, the positive CR deviations are additionally influenced by the artificial overshoot effects resulting from the application of the 21-day smooth (high-pass) filter, as demonstrated in Figure 3d. Significant increases in the CR flux are also evident at $t_{+10}$ and $t_{+23}$ which are due to the unintentional inclusion of GLEs in the composite (which were only filtered within a $\pm7$ day period around $t_{0}$).

Cloud anomalies (Fig. 8b) showed no clear response over the composite period: the largest negative/positive deviations occurred prior to the statistically significant CR flux variations, at t −26 (−0.39 ± 0.38%) and t −18 (0.33 ± 0.32%), corresponding to deviations of 2.5 ± 2.4σ and 2.1 ± 2.1σ respectively. The p-value associated with these mean anomalies is p = 0.01 (with upper/lower SEM values of p = 0.91/p < 0.00) and p = 0.04 (p < 0.00/p = 0.96) respectively. All other variations shown in Figure 8, including those observed following the Fd induced CR flux reduction, were of a smaller magnitude. The second largest negative anomaly observed occurred at t +5 and warrants discussion (for reasons which will become apparent in the next paragraph): the anomaly possesses a mean of −0.37 ± 0.41%, corresponding to a deviation of 2.4 ± 2.6σ and a significance of p = 0.02 (with upper/lower SEM values of p = 0.84/p < 0.00).

It should be stressed that by analyzing 81 points ($t\pm40$) over a composite using the $p = 0.05$ significance level, there will be a false discovery rate (FDR) of approximately $4$ days ($81 \times 0.05 = 4.05$) and we can expect that the mean anomaly will exceed the $p = 0.05$ level by chance on approximately four occasions over an 81-day period. Indeed, in Figure 8b we observe mean anomalies of $p \lt 0.05$ in six instances over the composite, in line with the expected FDR. In contrast, the CR flux is observed to be significant at the p = 0.05 level at 26 days out of 81 as a result of the influence of Fd and GLE events. We stress that the FDR noted here relates to the frequency with which the mean anomaly will appear as significant and not the error around the mean anomaly (SEM). These error intervals of course relate to how accurately the mean value is known, so in our case we have presented intervals which show with a ±95% accuracy the range in which the mean value is likely to occur. Consequently, the SEM ranges are regularly seen to pass the p = 0.05 threshold. Evaluating the statistical significance of both the mean and its error range will give a more robust indication of statistical significance than can be obtained from the mean value alone. For example, the $t_{−3}$ and $t_{0}$ CR flux anomalies are unambiguously statistically significant (SEM are $p \lt 0.00$), however, although the t −26 cloud anomalies show a mean and lower SEM which are statistically significant ($p \lt 0.05$), almost all of the upper SEM values exist at $p \gt 0.05$, indicating that this is not a robust result, as should be expected from a data point with its significance attributed to the FDR. Considered together these results clearly show that there are no unusual variations during or following statistically significant variations in the CR flux, and consequently support the rejection of $H_{1}$ and the acceptance of $H_{0}$.

From a combination of the observed reduction in CR flux of 3%, and the observation that no clear cloud changes occurred above the $p = 0.01$ significance level which are equivalent to cloud anomalies of about 0.40% (see Fig. 8b), we may also conclude that if a CR-cloud relationship exists, then a 1% CR flux change is, at most, able to alter cloud by ≤0.13% (0.4%/3%). If a CR flux-cloud relationship were more efficient than this limit, we would be able to detect a statistically significant cloud response over daily timescales. Since we do not, our conclusions must be limited by the statistical noise of the dependent (cloud) dataset. However, we note that supporting lines of evidence suggest at least that the higher range of values associated with this upper-limit constraint is likely too large, as it implies that over a solar cycle decadal oscillations in the CR flux on the order of 20% may induce cloud changes of ≤2.6% (20% × 0.13%, assuming a linear CR-cloud relationship), but no such periodic variations in cloud have been identified in either ISCCP or MODIS cloud data at any atmospheric level over the past 26 years of satellite observations (Kristjánsson et al. 2004; Agee et al. 2012; Laken et al. 2012b, 2012c).

Although the MC methods as shown in the previous examples have many advantages over classical statistical tests (like the Student’s t-test), we reiterate that there are situations where MC methods could yield incorrect or imprecise estimates of significance levels. Specifically, this may occur in instances where: (1) there is a limited amount of data to generate unique random samples. The total number of unique samples which may be generated can be calculated as

$MC_{sims} = \frac{m!}{n!(m-n)!}$

where $MC_{sims}$ is the number of unique simulations, $n$ is the subsample size (i.e., size of composites), and $m$ is the size of the parent population. If the number of unique samples is smaller than the number of simulations required for the MC to converge, then the accuracy of the MC-analysis method will be limited, (2) the number of MC simulations is quite low, resulting in a higher uncertainty associated with any calculated confidence intervals as the MCs had not fully converged (a cause of this may be either from point 1 or by experimental design); or (3) and the analyzed data contains outliers. This would be a problem only if point 1 and/or 2 were also occurring, thereby preventing the resulting MC distributions from becoming accurate representations of their parent population.

### 3.6. Common causes of false-positives ###

The data of Figure 8b clearly show no significant cloud response, and give no cause to support a hypothesized relationship between the CR flux and cloud cover. However, by constructing the composite in a subtly altered, yet still seemingly logical manner, highly different results can be produced. For example, if we seek to test how cloud cover varies following Fd events, it is reasonable to propose that we should compare cloud properties before Fd events to those following the events. Based on this, we may construct cloud anomalies against a supposedly undisturbed normalization period, prior to the occurrence of the Fd events, and then evaluate the significance of the changes by comparing the post-Fd event values to the values of the normalization period. While these propositions are reasonable, and this procedure has appeared in numerous publications (e.g., Kniveton 2004; Svensmark et al. 2009, 2012; Dragić et al. 2011, 2013), this approach can have a considerable impact on the statistical significance of the anomalies, as we will now demonstrate.

In Figure 9 we present cloud cover anomalies (%) calculated from the original ISCCP data (not adjusted for seasonality or mid-to-long-term variations), where the anomalies are defined in this instance by subtracting from the observed record the time average of the record over a five-day period starting at $t_{−10}$ (hereafter this five-day period is referred to as the normalization period). We note that these anomalies do not include the use of a 21-day smooth filter as in our previous examples. This different approach consequently produces values which differ from those presented in Figure 8b. Confidence intervals calculated from the normalization period alone and extended over the entire composite period (as opposed to calculating the confidence intervals at each $t$ point individually) based on 10,000 MC simulations at the $p = 0.05$ and $p = 0.01$ two-tailed levels are also presented in Figure 8 (red dashed and dotted lines respectively). In this procedure the time average of a five-day normalization period beginning at $t_{−10}$ has been subtracted from the observed record for 10,000 randomly generated composites, and the distribution of values within the normalization period has been accumulated to produce a PDF from which the confidence intervals were calculated. Relative to the normalization period of undisturbed conditions, we observe a reduction in cloud cover of −0.42% centered on $t_{+5}$ (a $3.5\sigma$ deviation with a statistical significance of $p \lt 0.01$). A traditional statistical test also indicates that this anomaly is highly significant: when we compare the cloud anomalies of the normalization period against the anomalies of t +5 using a Student’s $t$-test, we obtain a T-statistic of 2.87, corresponding to a $p$-value of 0.004.

In [ ]:
dshort = get_datelist(key_dates=fd_events, period=10)
xvals, crf_normed, crfErr_normed = composite(time_series=df.CRF, date_dic=dshort, norm_prd=[-10,-9,-8,-7,-6])
xvals, cld_normed, cldErr_normed = composite(time_series=df.cld, date_dic=dshort, norm_prd=[-10,-9,-8,-7,-6])

In [ ]:
%%time
ciNorm = mc_confidence_intervals(dataset=df.cld, k=44, its=5000, prd=10, norm_prd=[-10,-9,-8,-7,-6])

In [ ]:
fig9 = figure(width=450, plot_height=600, title=None, tools=TOOLS)
fig9.y_range = Range1d(start=-1,end=1)
fig9.yaxis.axis_label = "σ cloud cover (%)"
cldNorm_x, cldNorm_y = make_patch(x=xvals, y=cld_normed, yerr=cldErr_normed)

fig9.patch(cldNorm_x, cldNorm_y, color='#9999ff', fill_alpha=0.4, line_alpha=0.4)
fig9.line(xvals, cld_normed, color='black')

fig9.line(ciNorm['xvals'], ciNorm['p975'],color='blue', line_width = 1.0, line_dash=[3])
fig9.line(ciNorm['xvals'], ciNorm['p995'],color='blue', line_width = 1.0, line_dash = [1])
fig9.line(ciNorm['xvals'], ciNorm['p025'],color='blue', line_width = 1.0, line_dash=[3])
fig9.line(ciNorm['xvals'], ciNorm['p005'],color='blue', line_width = 1.0, line_dash=[1])

fig9.extra_y_ranges = {"foo": Range1d(start=-5, end=1)}
fig9.add_layout(LinearAxis(y_range_name="foo", axis_label="σ cosmic ray flux (%)"), 'right')
fig9.line(xvals, crf_normed, color='black', line_dash=[3], y_range_name='foo',line_width=2.0,alpha=0.4)

line1 = Span(location=ciNorm['norm_sigma']*1.89, dimension='width', 
             line_color='red', line_width=0.75, line_dash=[3])
line2 = Span(location=(ciNorm['norm_sigma']*1.89)*-1, dimension='width', 
             line_color='red', line_width=0.75, line_dash=[3])
line3 = Span(location=ciNorm['norm_sigma']*3.5, dimension='width', 
             line_color='red', line_width=0.75, line_dash=[1])
line4 = Span(location=(ciNorm['norm_sigma']*3.5)*-1, dimension='width', 
             line_color='red', line_width=0.75, line_dash=[1])
fig9.renderers.extend([line1,line2,line3,line4])

fig9.yaxis.axis_label_text_font_size = '10'
fig9.xaxis.axis_label = "Days since FD event"
fig9.xaxis.axis_label_text_font_size = '10'

show(fig9)

**Figure 9 ** Composite means of cloud cover anomalies (%, black solid line, left-hand axis) calculated by subtracting from the observed record the time average of the record over a five-day period starting at $t_{−10}$ (normalization period) and the corresponding CR flux anomalies (%, black dashed line, right-hand axis) processed in the same way. Based on MC simulations two sets of confidence intervals at the two-tailed $p = 0.05$ (dashed lines) and $p = 0.01$ (dotted lines) are calculated. The confidence intervals shown as red lines are calculated from a PDF of 10,000 MC simulations during the normalization period, while the confidence intervals shown as blue lines are calculated from PDFs of 10,000 MC simulations at each t individually (blue lines). Red lines indicate that the $t_{+5}$ cloud anomalies are $3.5\sigma$ ($p \lt 0.00$), while the lines show the same anomalies to be $1.89\sigma$ ($p = 0.06$).

However, the statistical significance of this result is incorrect. A comparison between the Fd composite values over t and the distribution of MC-generated composites only has meaning if the same statistic is compared. If the correct statistical methods are applied and confidence intervals are calculated for each t point (blue dashed and dotted lines in Fig. 9), the t +5 anomalies are found to have a p-value of 0.06, similar to the value obtained in Figure 8b. The false positive previously identified was a consequence of the violated assumption that variations over the composite are random and non-sequential (i.e., no autocorrelation). This assumption is generally untrue of geophysical data, and in the case of Figure 9, this is further exacerbated by the subtraction of the normalization period from the observed record.

Due to the temporal development of the weather, it is logical to state that the cloud conditions of today are more likely to be similar to those of yesterday than they are of last week. Consequently, any statistical tests, which compare the values of a normalization period against a possible peak (e.g., t +5) or any subsequent values, are inherently biased and will frequently produce false-positive results (this is the reason the t-test gave a false-positive result). In order to correctly assess statistical significance over t in a composite where a normalization period has been subtracted from the data, confidence intervals must be calculated at each t individually (e.g., the blue lines of Fig. 9). For each t the confidence intervals are drawn from a PDF of 10,000 MC simulations, which have been treated in an identical manner to the Fd composite (i.e., the randomly generated composites have had a five-day normalization period commencing at t −10 subtracted from the observed mean at each t). From the resulting confidence intervals we clearly demonstrate that the variations of t +5 are non-significant (p > 0.05).

We note that methodological differences in generating the anomalies between Figures 8 and 9 (black lines in both figures) have resulted in the t +5 decrease being exaggerated by 0.05% in the later figure. Despite this, the anomaly appeared to be weakly significant (p = 0.02) in Figure 8 but not in Figure 9 (p = 0.06). This is because the autocorrelation effects present in the analysis of Figure 9 have also resulted in relatively wider confidence intervals. This effect is also combined with the different calculation of the t +5 anomalies, and as a result we obtain two differing p-values. So which is correct? This is an intentionally misleading question, as the two results are merely independent statements of the probability of obtaining values of an equal magnitude by chance when the data is treated in a certain manner, including, excluding, or ignoring various effects. When this result is interpreted objectively, examining both the mean and error, longer-term variations, and excluding as many impertinent aspects of the data as is possible (e.g., by restricting influence from autocorrelations and minimizing noise), we may readily reach the conclusion that the t +5 anomaly is not unusual over the composite.

If we were to make the unsupported assumption that there exists a connection between the t +5 variation in cloud and the Fd events, then we may interpret the data to suggest that an approximate change in the CR flux of 1% may lead to global cloud changes of 0.12% (0.37/3.02, where 0.37 is the absolute cloud anomaly at t +5 in Fig. 8b). This response is even larger than our previously discussed upper limit value of 0.09% based on Svensmark et al. (2009). Objectively considered, there is no evidence yet presented for accepting that the t +5 anomaly is more than stochastic variability, or for making the assumption of a connection between the Fd events and cloud anomalies at t +5. However, it may always be favorably argued that the noise levels of the experiment may mask a small CR-cloud signal overwhelmed by noise. Indeed, experiments of this manner may only restrict the potential magnitude of such a signal by increasing experimental precision, but may never disprove it entirely.

To the previous point regarding autocorrelation in the data and its influence on the width and development of the confidence intervals over t we add some further remarks. If the composited cloud data of Figure 9 were to be replaced by an imaginary time series absent of autocorrelation (pure white noise), and these data were treated in the same manner (with a normalization period of five days), the resulting MC-calculated confidence intervals (equivalent to the blue lines of Fig. 9) would also display some weak non-stationary characteristics similar to those of Figure 9. The confidence intervals would be relatively narrow around the normalization period and expand (to a nearly static value) outside of the normalization period. The presence of persistent noise enhances these characteristics, resulting in narrower confidence intervals during the normalization period, which then expand with increasing time from the normalization period. The duration of this expansion and the final width of the confidence intervals are related to the amount of persistent long-memory noise within the data.

We again reiterate that traditional statistical tests (such as the Student’s t-test) may also be used to calculate significance after an adjustment in the assumed degrees of freedom (Forbush et al. 1982, 1983), which can be done by calculating the effective sample size (Neal 1993; Ripley 2009). We note that the MC-approach of the red lines in Figure 8 and the blue lines of Figure 9 is identical, and because the confidence intervals are calculated for each time-point individually, it is able to correctly account for the effects of restricted sample variability and autocorrelation present in the data and exacerbated by the normalization procedure.

## 3.7. A note on adding dimensions ##

The composites discussed thus far only concern area-averaged (one-dimensional) data. Such composites are usually from either point measurements (e.g., Harrison et al. 2011) or area-averaged variations (e.g., Svensmark et al. 2009) with time as the considered dimension. A limitation of this approach is that it does not provide the capacity to differentiate between small changes over large areas, or large changes over small areas: i.e., one-dimensional composites of this nature only enable an evaluation of integrated anomalies.

By considering the data at higher dimensions (i.e., over both temporal and spatial domains), differentiation between localized high-magnitude anomalies, and low-magnitude large-area anomalies is possible. However, the increased complexity also requires increased caution, as a dataset with three dimensions (e.g., latitude, longitude, and time) is essentially a series of parallel, independent hypothesis tests (see the description in the auxiliary material of Čalogović et al. 2010). To such data, the methods of one-dimensional composite analyses previously described may be applied. Following proper construction of the composite anomalies, the area over which anomalies are statistically significant may then be evaluated; this may be done by identifying a threshold p-value, and assessing the significance of the data points independently. Summing the number of statistically significant data points at each t of the composite gives a simple integer measure, against which the normality of the anomalies may be evaluated: this is referred to as the field significance (for further details, see Storch 1982; Livezey & Chen 1983; Wilks 2006).

If autocorrelation effects and the presence of factors influencing the data are absent (i.e., for an idealized sequentially independent random dataset), the percentage of significant pixels over the integrated field should only reflect the false discovery rate (FDR) associated with the chosen p-value. However, as previously noted, autocorrelation effects are normally a common feature of geophysical data. Cloud data show both spatial and temporal autocorrelation, and thus the number of significant data points at any given time is likely to be greater than expected from calculations of the FDR alone. The field significance may be effectively assessed via a variety of approaches such as the Walkers test (Wilks 2006). In addition, MC-approaches similar to those previously described in this work could be readily adapted to calculate distributions of randomly generated field significance against which to calculate a p-value.

While integrated (one-dimensional) composites are only able to test the net amplitude of anomalies, multi-dimensional composites may identify the extent of significant anomalies by creating a series of independent parallel hypothesis tests. However, a limiting weakness of such an approach, as previously noted, is the constraint of small sample areas on the signal-to-noise ratio (Fig. 5); this makes it less likely that a low-amplitude signal may be reliably detected over a small area. Therefore, at least in the context of studies discussed in this work, we suggest that multi-dimensional studies should be used in conjunction with one-dimensional time series analyses; e.g., to demonstrate that any statistical significance identified in one-dimensional tests does not result from localized high-amplitude noise.

## 3.8. Estimating statistical significance in subpopulations ##

The MC-based estimation of statistical significance we have described in this work has numerous advantages over traditional significance testing methods. However, it is not without limitations as previously stated at the end of Section 3.5. To these limitations we add a further caveat: MC-methods may provide inaccurate results where composites are constructed out of subpopulations of an original (parent) dataset. In the MC experiments presented in this work, we have assumed that the chances of any data point of a time series being included in a MC are equal. This is true, as the chance of a Fd event occurring is essentially random with respect to Earths atmosphere (although to this point we note that Fd occurrence tends to cluster around the period of solar maximum). However, it is not unusual for composite samples to be further constrained, adding additional selection criteria to composite events. Consequently, the resulting composites are created from subpopulations of the parent dataset, and thus their significance may not effectively be assessed by drawing random samples from the parent dataset.

In Section 3.3 we describe several studies where composites were restricted by Fd intensity, a common analysis approach. To continue with this example, imagine we were to restrict the n = 55 Fd events described in Section 3.3, to the most intense (I Fd ≥10%) n = 6 events and we wished to identify the p-value of the t 0 composite mean. To properly account for the sample restriction using a MC approach would require the creation of numerous n = 55 samples from the parent dataset (we shall refer to these samples as parent composites). Importantly, each of the parent composites needs to have the correct statistical properties, e.g., in this instance, a mean at t 0 comparable to that of the composite prior to restriction (i.e., the n = 55 Fd composite at t 0). MC populations would then need to be constructed from n = 6 subsamples, drawn from the parent composites. We may then use PDFs of the MC results to identify the p-value of the t 0 mean for the I Fd ≥ 10% composite.

This change in methodology is necessary as the hypothesis test is no longer concerned with determining the chance of obtaining a t 0 mean of n = 6 randomly, but rather, it is now concerned with determining the chance of obtaining a t 0 mean from n = 6 subpopulation when you begin with a parent composite of n = 55 with a specific mean value at t 0. i.e., the question becomes, if a group of events of n = 55 has a specific mean, what are the chances that a subsample of n = 6 of these events will have another specific mean?

# 4. Conclusions # 

Although numerous composite analyses have been performed to examine the hypothesized link between solar activity and climate, widely conflicting conclusions have been drawn. In this paper we have demonstrated that the cause of these conflicts may relate to differences in the various methodological approaches employed by these studies and the evaluation of the statistical significance of their results. We find that numerous issues may affect the analyses, including: (1) issues of signal-to-noise ratios connected to spatio-temporal restrictions; (2) interference from variability in data at timescales greater than those concerning hypothesis testing, which may not necessarily be removed by accounting for linear trends over the composite periods; (3) normalization procedures which affect both the magnitude of anomalies in composites, and estimations of their significance; (4) the application of statistical tests unable to account for autocorrelated data and biases imposed by the use of improper normalization procedures.

Statistical methods for correctly assessing significance in composites taking into account effective sample sizes have been previously established (Forbush et al. 1982, 1983; Singh 2006; Dunne et al. 2012). To these procedures, we add the composite construction outlined in this work, and a further robust procedure for the estimation of significance based on a Monte Carlo approach.

## Acknowledgments ##

We kindly thank Beatriz González Merino (Instituto de Astrofísica de Canarias), Thierry Dudock de Wit (University of Orleans), Jefrey R. Pierce (Colorado State University), Joshua Krissansen-Totton (University of Auckland), Eimear M. Dunne (Finnish Meteorological Institute), and two anonymous referees for their helpful comments. The list of Forbush decrease and Ground Level Enhancements was obtained from http://www.ngdc.noaa.gov/stp/solar/cosmic.html. Cosmic ray data were obtained from the Solar Terrestrial physics division of IZMIRAN from http://helios.izmiran.rssi.ru. The ISCCP D1 data are available from the ISCCP Web site at http://isccp.giss.nasa.gov/, maintained by the ISCCP research group at the NASA Goddard Institute for Space Studies. This work recieved support from the European COST Action ES1005 (TOSCA).

### Extra: Hurst exponent ###

Method of examining autocorrelation in the data.

In [ ]:
def hurst(ts):
    """
    Returns the Hurst Exponent (https://en.wikipedia.org/wiki/Hurst_exponent) a the time 
    series vector (ts). A measure of autocorrelation in the time series.
    from (https://github.com/drquant/Python_Finance/blob/master/mean_reversion_tutorial.py)
    """
    lags = range(2, 100) # Create range of lag values, and make array of vars of lag diffs.
    tau = [np.sqrt(np.std(np.subtract(ts[lag:], ts[:-lag]))) for lag in lags]
    poly = np.polyfit(np.log(lags), np.log(tau), 1)       # Lin. fit estimate of Hurst Exp.
    return poly[0]*2.0                  # Return the Hurst exponent from the polyfit output

In [ ]:

print("Hurst exp raw cloud data:{0:3.2f}. High-pass filtered:{1:3.2f}".format(hurst(df.cld),
                                                                                hurst(df.cld_anm21)))
print("Hurst exp raw CRF data:{0:3.2f}. High-pass filtered:{1:3.2f}".format(hurst(df.CRF),
                                                                                hurst(df.CRF_21anm)))